In [3]:
import torch
import torch.nn as nn 
import torch.optim as optim 
import torchvision.transforms as transforms 
import torchvision.datasets as datasets
import torch.utils.data as data 
import matplotlib.pyplot as plt 

import os 
from tqdm import tqdm

In [4]:
# -------------------------------------
# Configuration and Hyperparameters   
#--------------------------------------

In [5]:
BATCH_SIZE = 64 
LEARNING_RATE = 0.001 
EPOCHS = 10 
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_PATH = "cor_model.pth"

In [6]:
# ----------------------------------
# Data Loading and Preprocessing 
# ----------------------------------

In [7]:
transform = transforms.Compose([
    transforms.Grayscale(), # Ensure 1 Channel (grayscale)
    transforms.ToTensor(), 
    transforms.Normalize((0.5, ), (0.5, ))  # Normalize to (-1, 1)
])

In [ ]:
# Load EMNIST dataset 
train_dataset = datasets.EMNIST(root="./datasets", split="letters", train=True, transform=transform, download=True)
test_dataset = datasets.EMNIST(root="./datasets", split="letters", train=False, transform=transform, download=True)

train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

In [10]:
# ----------------------------------
# Model Definition (CNN)
# ----------------------------------

In [11]:
class HandwritingCNN(nn.Module):
    def __init__(self, num_classes=27): 
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1), 
            nn.ReLU(), 
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1), 
            nn.ReLU(), 
            nn.MaxPool2d(kernel_size=2, stride=2), 

            nn.conv2d(64, 128, kernel_size=3, stride=1, padding=1), 
            nn.ReLU(), 
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fc_layers = nn.Sequential(
            nn.Linear(128 * 3 * 3, 256),
            nn.ReLU(), 
            nn.Dropout(0.5), 
            nn.Linear(256, num_classes),
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)  # Flatten 
        x = self.fc_layers(x)
        return x 

In [ ]:
# -----------------------------------------
# Training Function
# -----------------------------------------

In [14]:
def train_model(model, train_loader, criterion, optimizer, epochs=EPOCHS): 
    model.train()
    for epoch in range(epoch):
        total_loss = 0 
        correct = 0
        total = 0 

        loop = tqdm(train_loader, leave=True)
        for images, labels in loop:
            images, labels = images.to(DEVICE), labels.to(DEVICE)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

            loop.set_description(f"Epoch [{epoch + 1} / {epochs}]")
            loop.set_postfix(loss=total_loss/len(train_loader), acc=100.*correct/total)
            

In [15]:
# ------------------------------
# Evaluation Function 
# ------------------------------

In [16]:
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0 
    total = 0 

    with torch.no_grad():
        for images, labels in test_loader: 
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

    accuracy = 100 * correct / total
    print(f"\nTest Accuracy: {accuracy:.2f}%")
    return accuracy 

In [17]:

# --------------------------------
# Main Training Execution 
# --------------------------------

In [ ]:
model = HandwritingCNN().to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [ ]:
print("STARTING TRAINING...")

train_model(model, train_loader, criterion, optimizer)

In [ ]:
# Save model 
torch.save(model.stat_dict(), MODEL_PATH)

In [ ]:
# Evaluate model
print(f"Model saved at {MODEL_PATH}")
evaluate_model(model, test_loader)

In [19]:
# ------------------------------------ 
#  Load and use the pretrain model
# ------------------------------------

In [ ]:
trained_model = HandwritingCNN()
trained_model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
trained_model.to(DEVICE)
trained_model.eval()

In [20]:
# Preprocess image 
transform = transforms.Compose([
    transforms.Grayscale(), 
    transforms.ToTensor(), 
    transforms.Normalize((0.5, ), (0.5, ))
])

In [21]:
def predict(image_path):
    image = Image.open(image_path).convert("L")
    image = transform(image).unsqueeze(0).to(DEVICE)

    with torch.no_grad():
        output = model(image)
        predicted_class = torch.argmax(output, dim=1).item()
    
    print(f"Predicted Character: {chr(predicted_class + 96)}")